# Jupyter notebook para el análisis exploratorio del datathon

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import re

In [ ]:
df_meteo = pd.read_csv('inputs/meteo_valencia.csv', sep=';')

In [ ]:
# Definición de los días festivos en la Comunidad Valenciana para el año 2019 y el primer mes de 2020
FESTIVOS = [
    '2019-03-19', '2019-04-19', '2019-04-22','2019-04-29', '2019-05-01', '2019-06-24', '2019-08-15','2019-10-09','2019-10-12','2019-11-01',
    '2019-12-06','2019-12-25','2020-01-01','2020-01-22'
]

## DATASET: Meteorología Valencia

### Corrección de errores de escritura 
En el dataset de meteorología de Valencia tanto las temperaturas como las precipitaciones tienen puntos y comas como delimitadores decimales. 
Lo que se hace es sustituir la coma por un punto para después poder transformar la columna a tipo float

In [ ]:
def change_decimal_delim(df_meteo: pd.DataFrame, columns: list) -> pd.DataFrame:
    for column in columns:
        df_meteo[column] = df_meteo[column].astype(str)
        df_meteo[column].replace({',':'.','nan':np.nan, ' ':np.nan, '':np.nan}, inplace=True, regex=True)
        df_meteo[column].fillna(0, inplace=True)
        df_meteo[column]= df_meteo[column].astype(float)
        df_meteo[column].replace(0.0, np.nan, inplace=True)
    return df_meteo
    
df_meteo = change_decimal_delim(df_meteo, ['Temp.', 'Precip.'])

Generación de una nueva columna de fecha (Date) en formato YYYY-MM-DD para poder unir después los dos datasets por la fecha

In [ ]:
dict_meses ={
    'Enero': '01', 'Febrero':'02', 'Marzo':'03', 'Abril':'04', 'Mayo':'05', 'Junio':'06', 'Julio':'07', 'Agosto':'08', 'Septiembre':'09',
    'Octubre':'10', 'Noviembre':'11', 'Diciembre':'12' 
}
df_meteo[['Year', 'Día']] = df_meteo[['Year', 'Día']].astype(str)
df_meteo['Mes'] = df_meteo['Mes'].apply(lambda x: dict_meses[x])
df_meteo['Date'] = df_meteo[['Year', 'Mes', 'Día']].agg('-'.join, axis=1)
df_meteo['Date'] = pd.to_datetime(df_meteo['Date'])

In [ ]:
# Se crea el dataframe de meteorología para Valencia tomando la media de la temperatura y las precipitaciones
# en todas las estaciones meteorológicas para cada día
df_meteo = df_meteo.groupby('Date')[['Temp.', 'Precip.']].mean()
df_meteo.reset_index(inplace=True)

In [ ]:
# Filtro para seleccionar las mismas fechas que en el dataset de Cajamar
df_meteo = df_meteo[(df_meteo['Date']>='2019-02-01') & (df_meteo['Date']<='2020-01-31')]
df_meteo.fillna(0, inplace=True)

### Relación entre las precipitaciones y la temperatura en Valencia. 
Se puede apreciar que no hay excesivas precipitaciones en la Comunidad Valenciana y parece que el hecho de que la temperatura aumente hace que las precipitaciones bajen puesto que en los meses estivales las precipitaciones son mucho menores

In [ ]:
fig = go.Figure()
# add line / trace 1 to figure: Temperatura
fig.add_trace(go.Scatter(
    x=df_meteo['Date'],
    y=df_meteo['Temp.'],
    marker=dict(
        color="blue"
    ),
    showlegend=False
))

# add line / trace 2 to figure: Precipitaciones
fig.add_trace(go.Scatter(
    x=df_meteo['Date'],
    y=df_meteo['Precip.'],
    marker=dict(
        color="green"
    ),
    showlegend=False
))
fig.update_layout(
    title="Distribución de la temperatura y las precipitaciones",
    xaxis_title="Date",
    yaxis_title="Precipitaciones y temperatura"
)
fig.show()

## DATASET: Cajamar Demanda

El dataset contiene demandas desde 2019-02-01 hasta 2020-01-31

In [ ]:
df_demanda = pd.read_csv('inputs/Modelar_UH2022.txt', sep='|')

Hay dos contadores que tienen datos anómalos. Los contadores dan valores negativos, por lo tanto se descartan estos dos contadores para el analisis de la estimación de la demanda

In [ ]:
df_demanda.groupby('ID')[['READINGINTEGER']].sum().sort_values(by='READINGINTEGER')
df_demanda = df_demanda[~df_demanda['ID'].isin([1041, 2711])]

In [ ]:
# Generacion de dos nuevas columnas que son la separación de la fecha en fecha y hora
df_demanda[['Date', 'Hour']] = df_demanda['SAMPLETIME'].str.split(' ', 1, expand=True)
df_demanda['Date'] = df_demanda['Date'].astype('datetime64')
df_demanda.drop(['SAMPLETIME'], axis=1, inplace=True)

In [ ]:
# Columna de decimales a integer para quitarle los .0 y luego se los pongo yo manualmente a todos. Despues se crea una sola
# columna de lectura real del contador

def clean_cols(df: pd.DataFrame, coltoclean:list) -> pd.DataFrame:
    if coltoclean == 'DELTATHOUSANDTH':
        df_demanda[coltoclean] = df_demanda[coltoclean].apply(lambda x: re.sub(r'[^0-9.]', '', str(x)))
        df_demanda[coltoclean].fillna('0', inplace=True)
    else:
        df_demanda[coltoclean].fillna(0.0, inplace=True)
    df_demanda[coltoclean] = df_demanda[coltoclean].astype(float).astype(int)
    df_demanda[coltoclean] = df_demanda[coltoclean].apply(lambda x: f"0.{x}")
    df_demanda[coltoclean] = df_demanda[coltoclean].astype(float)
    return df_demanda

df_demanda = clean_cols(df_demanda, 'READINGTHOUSANDTH')
df_demanda['lectura_contador'] = df_demanda['READINGINTEGER'] + df_demanda['READINGTHOUSANDTH']
df_demanda = clean_cols(df_demanda, 'DELTATHOUSANDTH')
df_demanda['consumo_calculado']= df_demanda['DELTAINTEGER'].astype(int) + df_demanda['DELTATHOUSANDTH']

### Diferencias entre consumos reales y calculados
Para poder determinar que columna utilizar como estimador se observan las diferencias entre el consumo real diario (lectura de los contadores) y el consumo calculado

In [ ]:
df_demanda = df_demanda.sort_values(by=['Hour'])
df_contadores = df_demanda.groupby(['ID', 'Date'])['lectura_contador'].agg(['first','last']).reset_index()
df_contadores['consumo_real'] = df_contadores['last'] - df_contadores['first']
df_calculados = df_demanda.groupby(['ID', 'Date'])['consumo_calculado'].sum().reset_index()

In [ ]:
df_consumos = pd.merge(df_contadores, df_calculados, on=['ID', 'Date'], how='inner')
df_consumos['Diferencias_consumo'] = df_consumos['consumo_real'] - df_consumos['consumo_calculado']

### Eliminar valores negativos o valores sin sentido de consumo
Hay varios casos:
- __Caso 1__: El valor de la lectura del contador es negativo 
    - __Caso 1a__: El valor del consumo calculado es positivo: se sustituye en el real
    - __Caso 1b__: El valor del consumo calculado es negativo: se restringe a cero el valor real para no tener valores negativos
- __Caso 2__ : El valor de la lectura del contador y el calculado son -1.00: se restringen a cero para no tener valores negativos  

In [ ]:
# Caso 2
df_consumos.loc[(df_consumos['consumo_real']==-1.00) & (df_consumos['consumo_calculado']==-1.00), 'consumo_real'] = 0.00
df_consumos.loc[(df_consumos['consumo_real']==-1.00) & (df_consumos['consumo_calculado']==-1.00), 'consumo_calculado'] = 0.00

# Caso 1b
df_consumos['consumo_real'] = np.where((df_consumos['consumo_real']<0) & (df_consumos['consumo_calculado']>0),
df_consumos['consumo_calculado'], df_consumos['consumo_real'])

# Caso 1a
df_consumos.loc[(df_consumos['consumo_real']<0), 'consumo_real'] = 0.00

In [ ]:
df_cajamar = pd.merge(df_meteo, df_consumos[['Date', 'ID', 'consumo_real', 'consumo_calculado']])

In [ ]:
fig = go.Figure()
# add line / trace 1 to figure: Temperatura
fig.add_trace(go.Scatter(
    x=df_cajamar['Date'],
    y=df_cajamar['Temp.'],
    marker=dict(
        color="blue"
    ),
    showlegend=False
))

# add line / trace 2 to figure: Precipitaciones
fig.add_trace(go.Scatter(
    x=df_cajamar['Date'],
    y=df_cajamar['Precip.'],
    marker=dict(
        color="green"
    ),
    showlegend=False
))
fig.add_trace(go.Scatter(
    x=df_cajamar['Date'],
    y=df_cajamar['consumo_real'],
    marker=dict(
        color="yellow"
    ),
    showlegend=False
))
fig.update_layout(
    title="Distribución de la temperatura y las precipitaciones",
    xaxis_title="Date",
    yaxis_title="Precipitaciones y temperatura"
)


fig.show()


In [ ]:
# Grafica del consumo real haciendo la suma de todas las estaciones de medida. 
# Haciendo la media queda la misma grafica casi y con la mediana se estabiliza todo pero no tiene mucho sentido aplicar la mediana
df = df_cajamar.groupby(['Date'])['consumo_real'].sum().reset_index()
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['Date'],
    y=df['consumo_real'],
    marker=dict(
        color="yellow"
    ),
    showlegend=False
))
fig.show()

## Clustering:

Ejercicios de agrupación de estaciones para ver cuales son las más similares entre ellas en consumo

In [ ]:
clusters = df_cajamar.groupby(['ID'])['consumo_real'].sum().reset_index()
fig = px.histogram(clusters, x="consumo_real")
fig.show()

# Hay 3 grupos claros de tipos de contadores:
     # 0-100K, 100K-300K, >300K
# Donde el grupo mayoritario es el primero y se podría segmentar en varios grupos 

In [ ]:
clusters['Cluster'] = np.where(
    clusters['consumo_real']<1e5,'1', 
        np.where((clusters['consumo_real']>=1e5)&(clusters['consumo_real']>=3e5), '2','3')
)

In [ ]:
cluster1 = clusters[clusters['Cluster']=='1']['ID'].tolist()
cluster2 = clusters[clusters['Cluster']=='2']['ID'].tolist()

df_cajamar['Cluster'] = np.where(
    df_cajamar['ID'].isin(cluster1),'1', 
        np.where(df_cajamar['ID'].isin(cluster2), '2','3')
)

## Corrección de outliers con Kats
Uso la libreria Kats para detectar outliers y para ver la estacionalidad de la serie.
Se analiza por separado cada cluster

In [ ]:
df_cluster1 = df_cajamar[df_cajamar['Cluster']=='1'].groupby(['Date'])['consumo_real'].sum().reset_index()
df_cluster2 = df_cajamar[df_cajamar['Cluster']=='2'].groupby(['Date'])['consumo_real'].sum().reset_index()
df_cluster3 = df_cajamar[df_cajamar['Cluster']=='3'].groupby(['Date'])['consumo_real'].sum().reset_index()

In [ ]:
from kats.utils.decomposition import TimeSeriesDecomposition
from kats.detectors.outlier import OutlierDetector
from kats.consts import TimeSeriesData

def create_ts(df: pd.DataFrame):
    # Construct TimeSeriesData object
    df = df[['Date','consumo_real']]
    df = df.rename(columns={"Date": "time", "consumo_calculado": "value"})
    ts = TimeSeriesData(df)
    return ts

def get_outliers(ts):
    outlier_detector = OutlierDetector(ts, "additive")
    outlier_detector.detector()
    outliers = outlier_detector.outliers
    ts_outliers = outlier_detector.remover(interpolate=True)
    print(outliers[0])
    return ts_outliers

def graph_wo_outliers(ts, ts_outliers):
    ax = ts.to_dataframe().plot(x="time", y="value")
    ts_outliers.to_dataframe().plot(x="time", y="y_0", ax=ax)
    plt.legend(labels=["original ts", "ts with removed outliers"])
    print(plt.show())

In [ ]:
ts1 = create_ts(df_cluster1)
ts2 = create_ts(df_cluster2)
ts3 = create_ts(df_cluster3)

In [ ]:
ts1_wo = get_outliers(ts1).to_dataframe()
ts2_wo = get_outliers(ts2).to_dataframe()
ts3_wo = get_outliers(ts3).to_dataframe()

In [ ]:
ts1_wo['Cluster'] = 1
ts2_wo['Cluster'] = 2
ts3_wo['Cluster'] = 3
df_preprocesed = pd.concat([ts1_wo, ts2_wo, ts3_wo], axis=0)
df_preprocesed.to_csv('inputs/Cajamar.csv', index=False)

## Visualización de los clusters

In [ ]:
df = df_cajamar.groupby(['Date', 'Cluster'])['consumo_real'].sum().reset_index()
fig = px.line(df,
    x='Date',
    y='consumo_real',
    color='Cluster'
)
fig.show()

# Quitando el cluster 2 que tiene valores mucho mas elevados. Los otros dos clusters tienen valores muy similares con un offset de un 40K

## Correlaciones entre variables
Se analiza la correlación de las variables de temperatura y precipitación con la variable objetivo

In [ ]:
df_preprocesed.columns = ['Date', 'consumo_real', 'Cluster']

In [ ]:
df_total = pd.merge(df_meteo, df_preprocesed, on='Date', how='inner')

In [ ]:
plt.figure(figsize=(10, 5))
mask = np.triu(np.ones_like(df_total.corr(), dtype=bool))
sns.heatmap(df_total.corr(),  annot=True, cmap='Dark2')